## Scraping Detik

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

In [2]:
# Mengecek Link status Ok
url = "https://news.detik.com/index"
response = requests.get(url)
response

<Response [200]>

In [3]:
links=[]
# jumlah Page yang diambil 
jumlah_index = 1

for index in range(jumlah_index):
    url = f"https://news.detik.com/indeks/{index+1}"  
    # print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('article', {"class": "list-content__item"})
    
    for article in articles:
        link = article.find('a')['href']
        links.append(link)
    
print("Total Links",len(links))

Total Links 20


In [4]:
title = []
author = []
date = []
content = []
category =[]

for link in links:
    url = link
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Judul Berita
    title_elem = soup.find('h1', {"class": "detail__title"})
    if title_elem:
        title_text = title_elem.text.strip()
    else:
        title_text = "Title not found"
    title.append(title_text)
    
    # Author berita
    author_elem = soup.find('div', {"class": "detail__author"})
    if author_elem:
        author_text = author_elem.get_text()
        author_text = author_text.split('-')[0].strip()
    else:
        author_text = "Author not found"
    author.append(author_text)
    
    # tanggal berita
    date_elem = soup.find('div', {"class": "detail__date"})
    if date_elem:
        date_text = date_elem.text.strip()
        
    else:
        date_text = "Date not found"
    date.append(date_text)
    
    # Category berita
    category_elements = soup.find('div',{"class": "page__breadcrumb"})
    if category_elements:
        category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
        category_text= category_text.text.strip()
    else:
        category_text = "Category not found"
    category.append(category_text)
   
    
    # Content Berita
    body_elem = soup.find('div', {"class": "detail__body"})
    
    if body_elem:
        content_elem = body_elem.find_all('p')
        content_text = ""
        for p in content_elem:
            content_text += p.text.strip() + "\n"
        
        if content_text.strip():
            content.append(content_text)
        else:
            content.append("Content not found")
    else:
        content.append("Content not found")
    
    

In [5]:
data = {'title': title,
        'author' : author,
        'category':category,
        'date': date,
        'content' : content,
        'link' : links
        }

df = pd.DataFrame(data)

df.head(10)

,title,author,category,date,content,link
0,PSI Angkat Giring Ganesha Jadi Dewan Pembina,Firda Cynthia Anggrainy,Pemilu,"Senin, 25 Sep 2023 19:39 WIB",Giring Ganesha diangkat pada jabatan baru seba...,https://news.detik.com/pemilu/d-6950217/psi-an...
1,"Calon Hakim Konstitusi Abdul Latif Diuji DPR, ...",Ari Saputra,Foto News,"Senin, 25 Sep 2023 19:34 WIB",Content not found,https://news.detik.com/foto-news/d-6950197/cal...
2,Tragis! Guru Aliyah di Demak Dibacok Murid Saa...,Mochamad Saifudin,Berita,"Senin, 25 Sep 2023 19:32 WIB",Seorang guru di Madrasah Aliyah di Kecamatan K...,https://news.detik.com/berita/d-6950210/tragis...
3,Pameran Mengenang Satu Tahun Tragedi Kanjuruhan,ANTARA FOTO/Ari Bowo Sucipto,Foto News,"Senin, 25 Sep 2023 19:30 WIB",Content not found,https://news.detik.com/foto-news/d-6949659/pam...
4,Kaesang Hadiri Acara Kopdarnas PSI di Djakarta...,Firda Cynthia Anggrainy,Pemilu,"Senin, 25 Sep 2023 19:24 WIB",Partai Solidaritas Indonesia (PSI) menggelar K...,https://news.detik.com/pemilu/d-6950192/kaesan...
5,9 dari 11 Wanita Pemeran Film Porno Jaksel Sud...,Wildan Noviansah,Berita,"Senin, 25 Sep 2023 19:23 WIB",Polda Metro Jaya mengidentifikasi ada 11 talen...,https://news.detik.com/berita/d-6950191/9-dari...
6,Viral Pria Bawa Motor Sambil Rebahan di Margon...,Devi Puspitasari,Berita,"Senin, 25 Sep 2023 19:18 WIB",Viral di media sosial video yang menunjukkan a...,https://news.detik.com/berita/d-6950173/viral-...
7,TNI: Mayat Pria Terbakar di Lanud Halim Bukan ...,Wildan Noviansah,Berita,"Senin, 25 Sep 2023 19:17 WIB",Seorang pria ditemukan tewas terbakar di pos a...,https://news.detik.com/berita/d-6950172/tni-ma...
8,Temuan Mayat Terbakar di Lanud Halim Perdanaku...,Wildan Noviansah,Berita,"Senin, 25 Sep 2023 19:10 WIB",Sesosok mayat pria ditemukan dalam kondisi ter...,https://news.detik.com/berita/d-6950164/temuan...
9,Alasan Pemerintah Larang Social Commerce: Lind...,Marlinda Oktavia Erwanti,Berita,"Senin, 25 Sep 2023 19:05 WIB",Menteri Komunikasi dan Informatika (Menkominfo...,https://news.detik.com/berita/d-6950160/alasan...


Cetak Simpan Data

In [6]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'./excel/detik_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')

Data has been saved to ./excel/detik_2023-09-25_19-40-19.xlsx


In [7]:
# Latihan Pengecek dan find element
url = f"https://news.detik.com/detiknews/d-6948823/siskaeee-penuhi-panggilan-polisi-soal-film-porno-ini-penampilannya"  
# print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")


# Category
# category_elements = soup.find('div',{"class": "page__breadcrumb"})
# print(category_elements)
# category_text = category_elements.find('a',{"dtr-sec": "breadcrumbkanal"})
# print(category_text.text.strip())

# ambil author detik
# title_elem = soup.find('div', {"class": "detail__author"})
# author_text = title_elem.get_text()
# author_name = author_text.split('-')[0].strip()
# print(author_name)